In [2]:
import keras
import numpy as np
from PIL import Image
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
# from misc import load_data1, load_data2, load_data3, load_data4, load_data5, load_data6, load_data7, load_data8
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D

In [3]:
# data = load_data1()

K.set_learning_phase(1)

sess = K.get_session()
X = K.tf.placeholder(shape=(None, 1818, 1228, 3), dtype=K.tf.float32)
Y = K.tf.placeholder(shape=(None, 5), dtype=K.tf.float32)

In [4]:
vgg = VGGFace(include_top=False, input_tensor = X, input_shape=(1228, 1818, 3))
for layer in vgg.layers:
    layer.trainable = False

In [5]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1228, 1818, 3)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 1228, 1818, 64)    1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 1228, 1818, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 614, 909, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 614, 909, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 614, 909, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 307, 454, 128)     0         
__________

In [83]:
p1 = Conv2D(filters=10, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p2 = Conv2D(filters=10, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=17).output)
p3 = Conv2D(filters=10, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p2))
p4 = Conv2D(filters=10, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p3))

# p11 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
# p12 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
# p21 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
# p22 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
# p31 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p3))
# p32 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p3)
# p41 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p4))
# p42 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p4)

# p11 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p12 = Conv2D(filters=5, kernel_size=3, strides=1, padding='same')(p2)
# p21 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p22 = Conv2D(filters=5, kernel_size=3, strides=1, padding='same')(p2)
# p31 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p3))
p32 = Conv2D(filters=5, kernel_size=5, strides=1, padding='same')(p3)
# p41 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p4))
p42 = Conv2D(filters=5, kernel_size=5, strides=1, padding='same')(p4)

In [85]:
# pre_boxes = ([K.reshape(p12, (-1, 4)), K.reshape(p22, (-1, 4)), K.reshape(p32, (-1, 4)), K.reshape(p42, (-1, 4))])
# pre_scores = ([K.reshape(p11, (-1, )), K.reshape(p21, (-1, )), K.reshape(p31, (-1, )), K.reshape(p41, (-1, ))])

pred = ([K.reshape(p12, (-1, 5)), K.reshape(p22, (-1, 5)), K.reshape(p32, (-1, 5)), K.reshape(p42, (-1, 5))])

In [66]:
# def filter_pred(scores = pre_scores, bbox = pre_boxes):
    
#     p12, p22, p32, p42 = bbox
    
#     p11_mask = K.greater_equal(scores[0], 0.5)
#     p21_mask = K.greater_equal(scores[1], 0.5)
#     p31_mask = K.greater_equal(scores[2], 0.5)
#     p41_mask = K.greater_equal(scores[3], 0.5)
    
#     p11 = K.reshape(K.tf.gather_nd(scores[0], indices= K.cast(p11_mask, dtype=K.tf.int32)), (-1, ))
#     p21 = K.reshape(K.tf.gather_nd(scores[1], indices= K.cast(p21_mask, dtype=K.tf.int32)), (-1, ))
#     p31 = K.reshape(K.tf.gather_nd(scores[2], indices= K.cast(p31_mask, dtype=K.tf.int32)), (-1, ))
#     p41 = K.reshape(K.tf.gather_nd(scores[3], indices= K.cast(p41_mask, dtype=K.tf.int32)), (-1, ))
    
#     p12 = K.reshape(K.tf.gather_nd(K.reshape(p12, (-1, 4)), indices= K.cast(p11_mask, dtype=K.tf.int32)), (-1, 4))
#     p22 = K.reshape(K.tf.gather_nd(K.reshape(p22, (-1, 4)), indices= K.cast(p21_mask, dtype=K.tf.int32)), (-1, 4))
#     p32 = K.reshape(K.tf.gather_nd(K.reshape(p32, (-1, 4)), indices= K.cast(p31_mask, dtype=K.tf.int32)), (-1, 4))
#     p42 = K.reshape(K.tf.gather_nd(K.reshape(p42, (-1, 4)), indices= K.cast(p41_mask, dtype=K.tf.int32)), (-1, 4))
    
#     return [p12, p22, p32, p42], [p11, p21, p31, p41], scores

In [67]:
# boxes, scores , sc = filter_pred()

In [147]:
def NMS(boxes = bb):
    
    Z0 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][0][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][0][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    Z1 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][1][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][1][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    Z2 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][2][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][2][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    Z3 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][3][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][3][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    
    A = K.reshape(K.concatenate(tensors= [K.tf.gather(boxes[0][0], Z0), K.tf.gather(boxes[0][1], Z1), K.tf.gather(boxes[0][2], Z2), K.tf.gather(boxes[0][3], Z3)], axis = 0), (-1, 5))

    return A

In [148]:
def NMS_2(A):
    Z0 = K.tf.image.non_max_suppression(boxes = K.reshape(A[0][:, 1:] , (-1, 4)), scores= K.reshape(A[0][:, 0], (-1, )),
                                        max_output_size = 25, iou_threshold=0.55)
    return K.gather(A[0], Z0)

In [149]:
sess.run(K.tf.global_variables_initializer())

In [150]:
b = sess.run([NMS()], feed_dict={X: np.array(Image.open("x.jpg")).reshape((1, 1818, 1228, 3))})

In [151]:
bb = sess.run([pred], feed_dict={X: np.array(Image.open("x.jpg")).reshape((1, 1818, 1228, 3))})

In [152]:
b_ = sess.run([NMS_2(b)], feed_dict={X: np.array(Image.open("x.jpg")).reshape((1, 1818, 1228, 3))})

In [153]:
b_

[array([[ 0.51799816,  0.09678266,  0.06544998, -0.10430518, -0.27147132],
        [ 0.46905053,  0.15218106,  0.2756063 ,  0.17617583, -0.18981919],
        [ 0.44489112,  0.1114768 ,  0.33546188,  0.21353637, -0.23104349],
        [ 0.44458875,  0.05728332,  0.23598811,  0.2399376 , -0.03817075],
        [ 0.43569475,  0.18146448,  0.10496051, -0.28053525, -0.18467803],
        [ 0.42700294,  0.23310378,  0.35930505,  0.06333331, -0.12676267],
        [ 0.3990336 ,  0.02420894, -0.00903574, -0.02190335, -0.31721956],
        [ 0.39735168,  0.13203952,  0.19403511, -0.14419506,  0.02438424],
        [ 0.39617842,  0.2110125 ,  0.24771284, -0.200387  ,  0.09360916],
        [ 0.3913962 , -0.23213181,  0.18845767,  0.20326889, -0.1068944 ],
        [ 0.38078538,  0.13627085,  0.30647066,  0.24982038, -0.31098965],
        [ 0.3662919 ,  0.14211276,  0.2910765 ,  0.19851546, -0.29776305],
        [ 0.35834008, -0.00790866, -0.01652671,  0.03181522, -0.38203365],
        [ 0.35705173, -0.

In [106]:
def sigmoid(X):
    return 1/(1 + K.exp(-X))

def relu(X):
    return K.maximum(X, 0)

def loss(Y, y_, a=0.6):
    if K.tf.size(Y) != K.tf.size(y_):
        y_ = y_[:2, :]
    softMax = K.mean(-(Y[:, 0] * K.log(sigmoid(y_[:, 0])) + ((1 - Y[:, 0]) * K.log(sigmoid(1 - y_[:, 0])))))
    mse = K.mean(keras.losses.mean_squared_error(Y[:, 1:], relu(y_[:, 1:])))
    return softMax + a*mse 

In [107]:
y_ = NMS(boxes, scores)
sess.run(K.tf.global_variables_initializer())
h = sess.run(fetches=y_, feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis=0)})

# Loss = loss(Y, y_)

ResourceExhaustedError: OOM when allocating tensor with shape[1,227,153,227,153,1]
	 [[Node: GatherNd_60 = GatherNd[Tindices=DT_INT32, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Sigmoid_19, Cast_52)]]
	 [[Node: Reshape_78/_785 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_100_Reshape_78", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'GatherNd_60', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/chawat/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/chawat/.local/lib/python3.5/site-packages/traitlets/config/application.py"

In [90]:
h.shape

(1, 227, 153, 1024)

In [91]:
opt = K.tf.train.AdagradOptimizer(learning_rate=0.0001)
train_ = opt.minimize(Loss)
sess.run(K.tf.global_variables_initializer())

NameError: name 'Loss' is not defined

In [10]:
def train(epochs = 10):
    for e in range(epochs):
        cost = 0
#         for i, d in enumerate(data):
        _, L = sess.run([train_, Loss], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis=0), Y: np.ones((2, 5)).reshape((-1, 5))})
        cost += L
        if e % 100 == 0:
            print("epoch {}, iter {} , loss {}".format(e, i , cost/len(data)))

In [ ]:
train(epochs = 12)

In [23]:
x = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])

In [28]:
s = np.array([0.43, 0.67, 0.5])

In [29]:
sm = np.greater_equal(s, 0.5)

In [30]:
x[sm]

array([[2, 3, 4],
       [3, 4, 5]])